### **마운트**


*   Google Colab에 마운트합니다.



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


### **라이브러리 및 모듈 로드**

*   필요한 라이브러리들을 전부 설치하고 모듈을 로드합니다.




In [ ]:
!apt-get update &> /dev/null
!apt install chromium-chromedriver &> /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin &> /dev/null
!pip install selenium &> /dev/null

from bs4 import BeautifulSoup
import requests
import urllib.request
from selenium import webdriver
import pandas as pd
import re
import time
import pickle

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

### **전역변수와 리스트 설정**

*   마지막 DataFrame에 들어갈 list들을 전역변수로 설정합니다.
*   키워드를 '저소득층'으로 설정하였고, 그 이유는 저소득층에 관련된 정책 혹은 지원, 시대별 저소득층 발생 원인 등에 대해서 알 수 있을 것 같아서 선정하였습니다.
*   추출 기간은 2011.01.01부터 2020.12.31입니다.





In [ ]:
url_list = []
title_list = []
year_list = []
month_list = []
day_list = []
content_list = []
news_list = []

keyword = '저소득층'
start_date = '20110101'
end_date = '20201231'

### **검색페이지 URL완성과 기사링크 접근하는 함수 만들기 + a) 제목, b) 날짜 추가**

*   검색 페이지 url을 위에 상기한 변수들을 통해 들어가고, 기사 링크 및 제목, 날짜까지 같이 크롤링합니다.
*   한 페이지에 기사가 10개가 출력되므로, for문을 통해 (총 기사 개수 / 10) + 1 번을 돌면서 필요한 정보들을 가져옵니다.
*   날짜는 날짜와 기사를 올린 시간이 묶여서 출력되는데, 날짜 부분만 가져오기 위해 [1:13]을 설정합니다.





In [ ]:
def get_infos(num):
    total_pages = int(num / 10) + 1

    driver = webdriver.Chrome('chromedriver', options=options)

    for i in range(1, total_pages + 1):
        url = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q={}&pg={}&sort=1&d1={}~{}'.format(keyword, i, start_date, end_date)
        driver.get(url)
        driver.maximize_window()

        try:
            for j in range(1, 10 + 1):
                title_url_xpath = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[2]/dl[{}]/dt/a'.format(j))
                date_xpath = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[2]/dl[{}]/dt/span'.format(j))

                title_list.append(title_url_xpath.text)
                url_list.append(title_url_xpath.get_attribute('href'))
                year_list.append(int(date_xpath.text[1:5]))
                month_list.append(int(date_xpath.text[7:9]))
                day_list.append(int(date_xpath.text[11:13]))
        
        except:
            break

    driver.close()


### **기사링크 크롤링하여 c) 본문을 리스트에 넣는 함수 만들기**

*   url_list에 있는 기사 링크들을 통해 본문의 내용을 가져옵니다.
*   tmp는 class를 art_body로 가지는 div를 가져오고, 그 tmp 내에서 find_all을 통해 본문을 담고 있는 부분을 가져옵니다. 반복문을 통해 tmp2 내에 있는 기사 본문들을 담고, 이를 content라는 변수를 통해 기사를 이어 붙이면서 리스트에 담습니다.



In [ ]:
def get_content(url_list, pickle_url):
    driver = webdriver.Chrome('chromedriver', options=options)

    crawling_counter = 1

    for url in url_list:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(2)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            tmp = soup.find('div', class_ = 'art_body')
            tmp2 = tmp.find_all('p', 'content_text')

            content = ''
            for i in tmp2:
                content += i.text
        
            pickle_url.append(content)
        
        except:
            pickle_url.append(' ')

        if crawling_counter % 100 == 0:
            print('{} out of {} content crawling done...'.format(crawling_counter, len(url_list)))

        crawling_counter += 1

    driver.close()


### **전체 기사의 개수 구하는 함수 만들기**

*   검색 페이지를 보면 해당 카테고리에 대해 총 몇개의 기사가 있는지 나옵니다.
*   메인 함수에서 제일 먼저 실행되는 구간입니다.



In [ ]:
def content_num(url):
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get(url)
    driver.maximize_window()

    num = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[2]/h3/span')
    
    return int(re.findall('\d+', num.text.replace(',', ''))[0])


### **메인함수 실행**

*   전체 기사의 개수를 구하고, 링크 / 제목 / 날짜를 가져오고, 마지막으로 본문을 가져오는 함수를 실행합니다.
*   각 항목에 대해서 진행 상황을 간단하게 print문으로 출력하였습니다.
* 함수 실행에 시간이 너무 많이 걸려서, pickle을 사용해 데이터를 쪼개고 다시 가져오는 작업을 거치게 됩니다.


In [ ]:
start = time.time()

init_url = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q={}&pg=1&sort=1&d1={}~{}'.format(keyword, start_date, end_date)

print('Get Number of Contents...')
num_content = content_num(init_url)
print('Total number of contents : {}'.format(num_content))
print()

print('Get Title, URL, Date...')
get_infos(num_content)
print('Title, URL, Date Crawling Done.')
print()

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_data.pickle', 'wb') as f:
    pickle.dump(url_list, f)
    pickle.dump(title_list, f)
    pickle.dump(year_list, f)
    pickle.dump(month_list, f)
    pickle.dump(day_list, f)

print('Total spend time : {} seconds'.format(time.time() - start))


Get Number of Contents...
Total number of contents : 5402

Get Title, URL, Date...
Title, URL, Date Crawling Done.

Total spend time : 839.7458550930023 seconds


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_data.pickle', 'rb') as f:
    url_list = pickle.load(f)
    title_list = pickle.load(f)
    year_list = pickle.load(f)
    month_list = pickle.load(f)
    day_list = pickle.load(f)

In [ ]:
start = time.time()

content_1_500 = []

print('Get Content number 1 to 500...')
get_content(url_list[:500], content_1_500)
print('Content number 1 to 500 Crawling Done.')

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_1_500.pickle', 'wb') as f:
    pickle.dump(content_1_500, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Get Content number 1 to 500...
100 out of 500 content crawling done...
200 out of 500 content crawling done...
300 out of 500 content crawling done...
400 out of 500 content crawling done...
500 out of 500 content crawling done...
Content number 1 to 500 Crawling Done.
Total spend time : 4607.638985872269 seconds


In [ ]:
start = time.time()

content_501_1000 = []

print('Get Content number 501 to 1000...')
get_content(url_list[500:1000], content_501_1000)
print('Content number 501 to 1000 Crawling Done.')

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_501_1000.pickle', 'wb') as f:
    pickle.dump(content_501_1000, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Get Content number 501 to 1000...
100 out of 500 content crawling done...
200 out of 500 content crawling done...
300 out of 500 content crawling done...
400 out of 500 content crawling done...
500 out of 500 content crawling done...
Content number 501 to 1000 Crawling Done.
Total spend time : 4687.345878362656 seconds


In [ ]:
start = time.time()

content_1001_1500 = []

print('Get Content number 1001 to 1500...')
get_content(url_list[1000:1500], content_1001_1500)
print('Content number 1001 to 1500 Crawling Done.')

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_1001_1500.pickle', 'wb') as f:
    pickle.dump(content_1001_1500, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Get Content number 1001 to 1500...
100 out of 500 content crawling done...
200 out of 500 content crawling done...
300 out of 500 content crawling done...
400 out of 500 content crawling done...
500 out of 500 content crawling done...
Content number 1001 to 1500 Crawling Done.
Total spend time : 4368.711542129517 seconds


In [ ]:
start = time.time()

content_1501_2500 = []

print('Get Content number 1501 to 2500...')
get_content(url_list[1500:2500], content_1501_2500)
print('Content number 1501 to 2500 Crawling Done.')

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_1501_2500.pickle', 'wb') as f:
    pickle.dump(content_1501_2500, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Get Content number 1501 to 2500...
100 out of 1000 content crawling done...
200 out of 1000 content crawling done...
300 out of 1000 content crawling done...
400 out of 1000 content crawling done...
500 out of 1000 content crawling done...
600 out of 1000 content crawling done...
700 out of 1000 content crawling done...
800 out of 1000 content crawling done...
900 out of 1000 content crawling done...
1000 out of 1000 content crawling done...
Content number 1501 to 2500 Crawling Done.
Total spend time : 12047.675377368927 seconds


In [ ]:
start = time.time()

content_2501_3500 = []

print('Get Content number 2501 to 3500...')
get_content(url_list[2500:3500], content_2501_3500)
print('Content number 2501 to 3500 Crawling Done.')

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_2501_3500.pickle', 'wb') as f:
    pickle.dump(content_2501_3500, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Get Content number 2501 to 3500...
100 out of 1000 content crawling done...
200 out of 1000 content crawling done...
300 out of 1000 content crawling done...
400 out of 1000 content crawling done...
500 out of 1000 content crawling done...
600 out of 1000 content crawling done...
700 out of 1000 content crawling done...
800 out of 1000 content crawling done...
900 out of 1000 content crawling done...
1000 out of 1000 content crawling done...
Content number 2501 to 3500 Crawling Done.
Total spend time : 17406.281531572342 seconds


In [ ]:
start = time.time()

content_3501_4500 = []

print('Get Content number 3501 to 4500...')
get_content(url_list[3500:4500], content_3501_4500)
print('Content number 3501 to 4500 Crawling Done.')

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_3501_4500.pickle', 'wb') as f:
    pickle.dump(content_3501_4500, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Get Content number 3501 to 4500...
100 out of 1000 content crawling done...
200 out of 1000 content crawling done...
300 out of 1000 content crawling done...
400 out of 1000 content crawling done...
500 out of 1000 content crawling done...
600 out of 1000 content crawling done...
700 out of 1000 content crawling done...
800 out of 1000 content crawling done...
900 out of 1000 content crawling done...
1000 out of 1000 content crawling done...
Content number 3501 to 4500 Crawling Done.
Total spend time : 20983.60771393776 seconds


In [ ]:
start = time.time()

content_4501_end = []

print('Get Content number 4501 to end...')
get_content(url_list[4500:], content_4501_end)
print('Content number 4501 to end Crawling Done.')

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_4501_end.pickle', 'wb') as f:
    pickle.dump(content_4501_end, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Get Content number 4501 to end...
100 out of 902 content crawling done...
200 out of 902 content crawling done...
300 out of 902 content crawling done...
400 out of 902 content crawling done...
500 out of 902 content crawling done...
600 out of 902 content crawling done...
700 out of 902 content crawling done...
800 out of 902 content crawling done...
900 out of 902 content crawling done...
Content number 4501 to end Crawling Done.
Total spend time : 15441.079872608185 seconds


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_1_500.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_501_1000.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_1001_1500.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_1501_2500.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_2501_3500.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_3501_4500.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/khan_content_4501_end.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

### **뉴스 리스트 작성**
* 어느 신문사에서 가져왔는지 알려주는 리스트입니다.

In [ ]:
news_list = ['경향'] * len(title_list)

### **저장한 리스트 DataFrame 작성**


*   각 list들을 DataFrame으로 묶는 구간입니다.



In [ ]:
total = pd.DataFrame({'title' : title_list, 'news' : news_list, 'year' : year_list, 'month' : month_list, 'day' : day_list, 'url' : url_list, 'content' : content_list})

### **불필요한 행(향이네 관련) DataFrame에서 제거**

*   경향신문 기사를 긁다 보면 간혹 '향이네'라는 곳의 기사가 가져와지는 경우가 있습니다.
*   http://h2.khan.co.kr/
*   '향이네'는 홈페이지의 구조가 기본 경향신문의 구조와 아예 다르고, 기사의 수도 매우 적습니다.
*   따라서 위의 본문에서 try, except문을 통해 에러가 발생하는 경우는 기사 본문을 ' '로 담도록 설정하였고, 그 행을 삭제하도록 하는 부분입니다.




In [ ]:
unused = []

for i in range(len(total['content'])):
    if total['content'][i] == ' ':
        unused.append(i)

total = total.drop(index = unused, axis = 0)
total = total.dropna(axis = 0)
total = total.reset_index(drop = True)
total

,title,news,year,month,day,url,content
0,[기고] 생리대를 무상 지급하라,경향,2020,12,31,http://news.khan.co.kr/kh_news/khan_art_view.h...,화장실에 들어갔다가 나와 남자 화장실인지 재차 확인했다. 화장실 안에 무료 생리대 ...
1,"2021 새해 이렇게 달라집니다…고교 전 학년 무상교육, 최저임금 시급 8720원",경향,2020,12,30,http://news.khan.co.kr/kh_news/khan_art_view.h...,내년부터 종합부동산세는 오르고 증권거래세는 낮아진다. 청약철회권 부여 등 금융소비자...
2,[기자칼럼]‘시선’에도 베인다,경향,2020,12,29,http://news.khan.co.kr/kh_news/khan_art_view.h...,"“다양한 신문 기사, 뉴스 단신, 라디오 프로그램이 고층 아파트의 삶을 담으려 했다..."
3,[특파원 칼럼]바이든 ‘기후변화 정치’ 딜레마,경향,2020,12,29,http://news.khan.co.kr/kh_news/khan_art_view.h...,"도널드 트럼프 미국 대통령에게 대표 정책이 ‘반이민’이었다면, 내년 1월20일 취임..."
4,혈액 수급 돕는 따뜻한 기프트카 ‘레드카펫 헌혈 캠페인’,경향,2020,12,29,http://news.khan.co.kr/kh_news/khan_art_view.h...,현대차그룹현대차그룹이 활발한 사회공헌활동을 통해 코로나19로 침체된 한국 사회에 따...
...,...,...,...,...,...,...,...
5344,“배고플 때 받은 자활의 떡 이젠 이웃들과 나눠야죠”,경향,2011,1,4,http://news.khan.co.kr/kh_news/khan_art_view.h...,사회적기업 꿈꾸는 떡집 ‘떡찌니’ 석창근씨 가족서울 도곡동의 떡집 ‘떡찌니’는 요즘...
5345,[사설]‘보편적 무상급식의 원년’으로 기록될 올해,경향,2011,1,3,http://news.khan.co.kr/kh_news/khan_art_view.h...,전국 16개 시·도 가운데 올해 ‘보편적 무상급식(무상급식)’을 실시하는 곳이 많다...
5346,"썬밸리골프그룹, 사랑을 나누다",경향,2011,1,3,http://news.khan.co.kr/kh_news/khan_art_view.h...,"세밑 한파가 기승을 부린 지난 주, 썬밸리골프그룹과 동광종합토건(주) (회장 이신근..."
5347,눈칫밥 없는 ‘무상급식 원년’ 새 이정표,경향,2011,1,2,http://news.khan.co.kr/kh_news/khan_art_view.h...,시·도별 어떻게 시작하나‘무상급식 원년’올해는 모든 학생이 차별 없이 무상으로 학교...


### **본문 및 제목 불필요한 내용 전처리**

*   먼저 영어, 숫자, 한글, 공백을 제외한 모든 기호를 제외합니다.
*   다음으로는 공백이 2번 이상인 경우가 있을 수 있다고 생각하여, 그것들을 모두 공백 한칸으로 처리합니다.
*   strip 함수를 통해 전체 content의 앞, 뒤로 있는 공백을 없앱니다.
*   처리가 잘 되었는지 확인을 위해 상위 5개의 결과값을 출력합니다.





In [ ]:
total['content'] = total['content'].str.replace(r'[^a-zA-Z0-9\s가-힣]', ' ')
total['content'] = total['content'].str.replace(r'[ ]{2,}', ' ')
total['content'] = total['content'].str.strip()

total['content'][:5]

0    화장실에 들어갔다가 나와 남자 화장실인지 재차 확인했다 화장실 안에 무료 생리대 바...
1    내년부터 종합부동산세는 오르고 증권거래세는 낮아진다 청약철회권 부여 등 금융소비자 ...
2    다양한 신문 기사 뉴스 단신 라디오 프로그램이 고층 아파트의 삶을 담으려 했다 그 ...
3    도널드 트럼프 미국 대통령에게 대표 정책이 반이민 이었다면 내년 1월20일 취임할 ...
4    현대차그룹현대차그룹이 활발한 사회공헌활동을 통해 코로나19로 침체된 한국 사회에 따...
Name: content, dtype: object

In [ ]:
total['title'] = total['title'].str.replace(r'[^a-zA-Z0-9\s가-힣]', ' ')
total['title'] = total['title'].str.replace(r'[ ]{2,}', ' ')
total['title'] = total['title'].str.strip()

total['title'][:5]

0                                 기고 생리대를 무상 지급하라
1    2021 새해 이렇게 달라집니다 고교 전 학년 무상교육 최저임금 시급 8720원
2                                  기자칼럼 시선 에도 베인다
3                          특파원 칼럼 바이든 기후변화 정치 딜레마
4                   혈액 수급 돕는 따뜻한 기프트카 레드카펫 헌혈 캠페인
Name: title, dtype: object

### **최소 5개 사례 print하여 제시**

*   head 함수를 통해 출력합니다. 저는 10개를 출력해 보았습니다.



In [ ]:
total.head(10)

,title,news,year,month,day,url,content
0,기고 생리대를 무상 지급하라,경향,2020,12,31,http://news.khan.co.kr/kh_news/khan_art_view.h...,화장실에 들어갔다가 나와 남자 화장실인지 재차 확인했다 화장실 안에 무료 생리대 바...
1,2021 새해 이렇게 달라집니다 고교 전 학년 무상교육 최저임금 시급 8720원,경향,2020,12,30,http://news.khan.co.kr/kh_news/khan_art_view.h...,내년부터 종합부동산세는 오르고 증권거래세는 낮아진다 청약철회권 부여 등 금융소비자 ...
2,기자칼럼 시선 에도 베인다,경향,2020,12,29,http://news.khan.co.kr/kh_news/khan_art_view.h...,다양한 신문 기사 뉴스 단신 라디오 프로그램이 고층 아파트의 삶을 담으려 했다 그 ...
3,특파원 칼럼 바이든 기후변화 정치 딜레마,경향,2020,12,29,http://news.khan.co.kr/kh_news/khan_art_view.h...,도널드 트럼프 미국 대통령에게 대표 정책이 반이민 이었다면 내년 1월20일 취임할 ...
4,혈액 수급 돕는 따뜻한 기프트카 레드카펫 헌혈 캠페인,경향,2020,12,29,http://news.khan.co.kr/kh_news/khan_art_view.h...,현대차그룹현대차그룹이 활발한 사회공헌활동을 통해 코로나19로 침체된 한국 사회에 따...
5,미 교육계 팬데믹 학습 격차 해소 골몰,경향,2020,12,29,http://news.khan.co.kr/kh_news/khan_art_view.h...,비대면 수업 등 부작용 우려에 학교 혁신의 기회로 삼자 교육단체 전문가 논의 활발코...
6,새벽 틈 타 읍사무소에 쌀 포대와 손편지 놓고간 천사,경향,2020,12,28,http://news.khan.co.kr/kh_news/khan_art_view.h...,금년 한해는 온통 세상이 코로나 역병으로 정말 살기 어려운 한 해 였어요 강추위가 ...
7,2021년 이렇게 달라집니다 교육 보육 가족 고교 무상교육 전면 확대 시행,경향,2020,12,28,http://news.khan.co.kr/kh_news/khan_art_view.h...,고2 3학년을 대상으로 시행 중인 고교 무상교육이 1학년까지 포함해 전면 확대 시행...
8,뉴욕 총기 피해 2배 급증 피해자 96 는 비백인,경향,2020,12,28,http://news.khan.co.kr/kh_news/khan_art_view.h...,미국 뉴욕시의 총기 사상자 수가 지난해보다 2배 이상 늘었다는 뉴욕 경찰의 조사 결...
9,저소득층 자동차 사고 피해 정부지원금 내년부터 2만원 인상,경향,2020,12,27,http://news.khan.co.kr/kh_news/khan_art_view.h...,기초생활수급자나 차상위계층이 자동차 사고로 중증 장애를 입었거나 사망할 경우 지급되...


### **CSV 파일로 저장**

*   google drive를 사용하여 제 드라이브의 폴더에 저소득층_경향이라는 파일 이름으로 csv 파일을 저장하였습니다.



In [ ]:
total.to_csv('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/저소득층_경향.csv', encoding="utf-8-sig")